In [30]:
import torch
import streamlit as st
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
import json
import os
from datetime import datetime
from typing import List, Dict, Optional

#### Ticket class to manage service tickets

In [36]:
class Ticket:
    """Represents a service ticket with issue and solution."""
    def __init__(self, ticket_id: str, title: str, description: str, 
                 category: str, status: str = "Open", solution: str = "", 
                 reasoning: str = "", timestamp: str = None, severity: str = "Medium",
                 application: str = "", affected_users: str = "", environment: str = "Production"):
        self.ticket_id = ticket_id
        self.title = title
        self.description = description
        self.category = category
        self.status = status
        self.solution = solution
        self.reasoning = reasoning
        self.timestamp = timestamp or datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.severity = severity
        self.application = application
        self.affected_users = affected_users
        self.environment = environment

    # Convert ticket to dictionary.
    def to_dict(self):
        
        return {
            "ticket_id": self.ticket_id,
            "title": self.title,
            "description": self.description,
            "category": self.category,
            "status": self.status,
            "solution": self.solution,
            "reasoning": self.reasoning,
            "timestamp": self.timestamp,
            "severity": self.severity,
            "application": self.application,
            "affected_users": self.affected_users,
            "environment": self.environment
        }
    
    # Create ticket from dictionary.
    @classmethod
    def from_dict(cls, data: dict):
        return cls(**data)

In [37]:
#Test Ticket class serialization
new_ticket = Ticket(
    ticket_id="TKT-0001",
    title="Payment API Returning 500 Errors",
    description="All transactions failing...",
    category="API Issues",
    status="Open",
    severity="Critical",
    application="Payment API",
    environment="Production"
)

# Serialize ticket to dictionary
ticket_dict = new_ticket.to_dict()
print(ticket_dict)

print("\n-Deserialize ticket from dictionary-\n")
# Deserialize ticket from dictionary
loaded_ticket = Ticket.from_dict(ticket_dict)
print(loaded_ticket.title) 

{'ticket_id': 'TKT-0001', 'title': 'Payment API Returning 500 Errors', 'description': 'All transactions failing...', 'category': 'API Issues', 'status': 'Open', 'solution': '', 'reasoning': '', 'timestamp': '2025-10-21 13:44:20', 'severity': 'Critical', 'application': 'Payment API', 'affected_users': '', 'environment': 'Production'}

-Deserialize ticket from dictionary-

Payment API Returning 500 Errors


#### TicketManager Class Functions to save and load tickets

In [34]:
class TicketManager:

    # Manages ticket storage and retrieval.
    
    def __init__(self, storage_file="tickets.json"):
        self.storage_file = storage_file
        self.tickets = self.load_tickets()
    
    def load_tickets(self) -> List[Ticket]:
        """Load tickets from JSON file."""
        if os.path.exists(self.storage_file):
            try:
                with open(self.storage_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    return [Ticket.from_dict(t) for t in data]
            except Exception as e:
                st.error(f"Error loading tickets: {e}")
                return []
        return []
    
    def save_tickets(self):
        """Save tickets to JSON file."""
        try:
            with open(self.storage_file, 'w', encoding='utf-8') as f:
                json.dump([t.to_dict() for t in self.tickets], f, indent=2, ensure_ascii=False)
        except Exception as e:
            st.error(f"Error saving tickets: {e}")
    
    def add_ticket(self, ticket: Ticket):
        """Add a new ticket."""
        self.tickets.append(ticket)
        self.save_tickets()
    
    def update_ticket(self, ticket_id: str, solution: str, reasoning: str, status: str = "Resolved"):
        """Update a ticket with solution."""
        for ticket in self.tickets:
            if ticket.ticket_id == ticket_id:
                ticket.solution = solution
                ticket.reasoning = reasoning
                ticket.status = status
                self.save_tickets()
                return True
        return False
    
    def get_tickets_by_severity(self, severity: str) -> List[Ticket]:
        """Get tickets filtered by severity."""
        return [t for t in self.tickets if t.severity == severity]
    
    def get_tickets_by_application(self, application: str) -> List[Ticket]:
        """Get tickets filtered by application."""
        return [t for t in self.tickets if t.application == application]
    
    def get_ticket(self, ticket_id: str) -> Optional[Ticket]:
        """Get a specific ticket."""
        for ticket in self.tickets:
            if ticket.ticket_id == ticket_id:
                return ticket
        return None
    
    def get_all_tickets(self) -> List[Ticket]:
        """Get all tickets."""
        return self.tickets
    
    def get_resolved_tickets(self) -> List[Ticket]:
        """Get only resolved tickets with solutions."""
        return [t for t in self.tickets if t.status == "Resolved" and t.solution]
    
    def generate_ticket_id(self) -> str:
        """Generate a unique ticket ID."""
        return f"TKT-{len(self.tickets) + 1:04d}"

In [27]:
#Test Ticket class serialization

manager = TicketManager()
manager.load_tickets() #← Loads tickets from JSON file

print("\n-Loaded Tickets-\n")
for t in manager.tickets:
    print(f"{t.ticket_id}: {t.title} - {t.status}")

# Save tickets back to file
manager.save_tickets()  #← Saves tickets to JSON file

# Get tickets by severity
print("\n-Tickets by Severity-\n")
critical_tickets = manager.get_tickets_by_severity("Critical")
for t in critical_tickets:
    print(f"{t.ticket_id}: {t.title} - {t.status}")



-Loaded Tickets-

TKT-0001: Payment API Returning 500 Internal Server Error - Resolved
TKT-0002: Dashboard Loading  Slow - Resolved
TKT-0003: Ppipeline Failiure - Open
TKT-0004: Data base connectivity issue - Open
TKT-0005: Dashboard Loading Extremely Slow - Resolved

-Tickets by Severity-

TKT-0001: Payment API Returning 500 Internal Server Error - Resolved
TKT-0002: Dashboard Loading  Slow - Resolved
TKT-0003: Ppipeline Failiure - Open


#### Functions to create vector store and find similar tickets

In [38]:
def create_ticket_embeddings(tickets: List[Ticket]):
    """Create vector embeddings from historical tickets."""
    if not tickets:
        return None
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2",
        model_kwargs={"device": device}
    )
    
    # Create text documents from tickets (title + description + solution)
    documents = []
    metadatas = []
    
    for ticket in tickets:
        text = f"Title: {ticket.title}\nDescription: {ticket.description}\nSolution: {ticket.solution}\nReasoning: {ticket.reasoning}"
        documents.append(text)
        metadatas.append({
            "ticket_id": ticket.ticket_id,
            "category": ticket.category,
            "title": ticket.title,
            "severity": ticket.severity

        })
    print(documents,metadatas)
        # Create FAISS vector store
    vector_store = FAISS.from_texts(texts=documents, embedding=embeddings, metadatas=metadatas)
    return vector_store
